In [ ]:
# Importing necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count
import matplotlib.pyplot as plt
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import Imputer
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
#Creating a SparkSession
spark = SparkSession.builder \
    .appName("TrafficVolumeAnalysis") \
    .getOrCreate()

In [ ]:
#Loading the dataset into a PySpark DataFrame
file_path = "file:///home/hduser/Desktop/SEM2_CA1/da_sem2_ca1_rep/dataset.csv"
df = spark.read.csv(file_path, header=True)

In [ ]:
# Displaying schema
df.printSchema()

In [ ]:
# Displaying first few rows
df.show(5)

In [ ]:
# Summary statistics
df.describe().show()

In [ ]:
# Dropping columns 
columns_to_drop = ['STATISTICS', 'TLIST(A1)', 'C02875V03459', 'UNIT']
df = df.drop(*columns_to_drop)

In [ ]:
# Displaying first few rows
df.show(5)

In [ ]:
# Summary statistics
df.describe().show()

In [ ]:
#Filtering thr rows as required
df = df.filter(df['Statistic Label'] == "Traffic Volume")

In [ ]:
df.show()

In [ ]:
# Checking for missing values
missing_counts = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])
missing_counts.show()

In [ ]:
# Extracting values for plotting
years = df.select("Year").rdd.flatMap(lambda x: x).collect()
traffic_volumes = df.select("VALUE").rdd.flatMap(lambda x: x).collect()

In [ ]:
# Plotting traffic volume over the years with logarithmic scale on Y-axis
plt.figure(figsize=(12, 6))
plt.plot(years, traffic_volumes, marker='o', linestyle='-')
plt.yscale('log')  # Set logarithmic scale for Y-axis
plt.title("Traffic Volume Over the Years")
plt.xlabel("Year")
plt.ylabel("Traffic Volume")
plt.grid(True)
plt.show()